In [1]:
import osmnx as ox
import pandas as pd
import os

In [2]:
pwd

'C:\\Users\\91628\\Tata-Hackathon-2025\\data'

In [3]:
ls

 Volume in drive C has no label.
 Volume Serial Number is F248-05BE

 Directory of C:\Users\91628\Tata-Hackathon-2025\data

29-07-2025  00:45    <DIR>          .
29-07-2025  00:45    <DIR>          ..
29-07-2025  00:29    <DIR>          .ipynb_checkpoints
29-07-2025  00:05    <DIR>          cache
28-07-2025  22:38             1,946 cleaned-EV-Dataset.csv
28-07-2025  23:19            65,404 ev_charging_stations_granular.json
28-07-2025  23:52            18,905 final_ev_stations.csv
28-07-2025  23:59            42,297 Granular-EVStationData.ipynb
29-07-2025  00:29             3,985 importing-map.ipynb
29-07-2025  00:22       188,952,073 road_network_Delhi_India.graphml
28-07-2025  17:20               760 RS_Session_266_AU_2960_B_i.csv
28-07-2025  17:15               768 RS_Session_267_AU_3266_2_0.csv
28-07-2025  22:25             1,105 RS_Session_267_AU_3924_A.csv
28-07-2025  17:22               128 RS_Session_267_AU_580_A.csv
28-07-2025  17:17             1,216 RS_Session_267_AU_581_C_i

In [7]:
# --- Load the Road Network Graph ---
data_folder = ''
graph_file_name = "road_network_Delhi_India.graphml"
graph_file_path = os.path.join(data_folder, graph_file_name)

In [9]:
G = ox.load_graphml(filepath=graph_file_path)

In [11]:
# ---  Load the Cleaned EV Stations Data ---
final_cleaned_file = 'final_ev_stations.csv'
# os.path.join('', 'final_ev_stations.csv') results in 'final_ev_stations.csv'
final_cleaned_file_path = os.path.join(data_folder, final_cleaned_file)

In [13]:
print(f"Loading df_stations from: {final_cleaned_file_path}")
df_stations = pd.read_csv(final_cleaned_file_path)
print(f"Loaded df_stations (Shape: {df_stations.shape})")

Loading df_stations from: final_ev_stations.csv
Loaded df_stations (Shape: (116, 11))


In [15]:
# --- Find the Nearest Graph Node for Each Charging Station ---
print("\n--- Finding nearest graph nodes for charging stations (this might take a moment)... ---")

points = list(zip(df_stations['latitude'], df_stations['longitude']))
nearest_nodes = []

for i, (lat, lon) in enumerate(points):
    # ox.distance.nearest_nodes expects (graph, x(longitude), y(latitude))
    nearest_node = ox.distance.nearest_nodes(G, lon, lat)
    nearest_nodes.append(nearest_node)
    if (i + 1) % 50 == 0 or (i + 1) == len(points):
        print(f"  Processed {i + 1}/{len(points)} stations.")

df_stations['nearest_node'] = nearest_nodes

initial_stations_count = df_stations.shape[0]
df_stations.dropna(subset=['nearest_node'], inplace=True)
stations_after_node_dropna = df_stations.shape[0]
if initial_stations_count != stations_after_node_dropna:
    print(f"Dropped {initial_stations_count - stations_after_node_dropna} stations that couldn't be aligned with the road network.")

df_stations['nearest_node'] = df_stations['nearest_node'].astype(int)

print("\n--- Successfully found nearest nodes! ---")
print(f"Shape of df_stations after aligning: {df_stations.shape}")

# --- 4. Final check and save ---
print("\n--- First 5 Rows with 'nearest_node' (df_stations.head()) ---")
display(df_stations.head())

output_aligned_file_name = 'aligned_ev_stations.csv'
output_aligned_file_path = os.path.join(data_folder, output_aligned_file_name)
df_stations.to_csv(output_aligned_file_path, index=False)
print(f"\n--- Saved aligned EV stations to {output_aligned_file_path} ---")

print("\n--- Alignment of charging stations with road network complete. ---")


--- Finding nearest graph nodes for charging stations (this might take a moment)... ---
  Processed 50/116 stations.
  Processed 100/116 stations.
  Processed 116/116 stations.

--- Successfully found nearest nodes! ---
Shape of df_stations after aligning: (116, 12)

--- First 5 Rows with 'nearest_node' (df_stations.head()) ---


,address,city,availability_status,latitude,longitude,station_name,pincode,pricing,state,charger_type_code,charger_type_details,nearest_node
0,EESL EDC Parking Goa,Goa,1,15.510472,73.837222,EESL Legislative Assembly Goa,403503,free,Goa,0,AC Charger (Slow),11524192253
1,"SDMC Parking, R Block, GK-1, DELHI-110016",Delhi,1,28.547217,77.244324,EESL R block GK1,110016,free,Delhi,0,AC Charger (Slow),6436645068
2,"SDMC Parking, SDA Market, Hauz Khas, New Delhi...",Delhi,1,28.546273,77.197110,EESL SDA market,110016,free,Delhi,0,AC Charger (Slow),4233845033
3,"Near Tata Advance Systems, sector- 59, Noida",Noida,1,28.608947,77.362725,EESL Tata Advance Systems Noida,201301,free,Uttar Pradesh,0,AC Charger (Slow),1820933814
4,"EESL Feroze Gandhi Road, Lajpat Nagar, SDMC pa...",Delhi,1,28.558750,77.090957,EESL Feroze Gandhi Road Lajpat Nagar,110024,free,Delhi,0,AC Charger (Slow),413760670



--- Saved aligned EV stations to aligned_ev_stations.csv ---

--- Alignment of charging stations with road network complete. ---
